# ollama

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/ollama.ipynb)

Like [vLLM](./vllm.ipynb), ollama is a fast and easy-to-use library for LLM inference and serving.

Docs:
- https://github.com/ollama/ollama
- https://ollama.com/

> <p style="color:red;">ollama should be installed and running.<p>

Download and install from https://ollama.com/download.

In [2]:
!ollama --version

ollama version is 0.9.6


In [3]:
!pip3 install -U ollama

  Using cached ollama-0.5.1-py3-none-any.whl.metadata (4.3 kB)
Using cached ollama-0.5.1-py3-none-any.whl (13 kB)


In [10]:
import ollama

In [6]:
ollama.list()

ListResponse(models=[])

In [7]:
# Pull the Gemma3n model
# https://ollama.com/search
ollama.pull('gemma3n:latest')

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [11]:
ollama.list()

ListResponse(models=[Model(model='gemma3n:latest', modified_at=datetime.datetime(2025, 7, 27, 11, 40, 47, 619716, tzinfo=TzInfo(+02:00)), digest='15cb39fd9394fd2549f6df9081cfc84dd134ecf2c9c5be911e5629920489ac32', size=7547589116, details=ModelDetails(parent_model='', format='gguf', family='gemma3n', families=['gemma3n'], parameter_size='6.9B', quantization_level='Q4_K_M'))])

In [12]:
import nest_asyncio
nest_asyncio.apply()

class Client:
    def __init__(self, model="gemma3n:latest"):
        self.client = ollama.AsyncClient()
        self.model = model
        self.messages = []

    async def chat(self, content: str):
        self.messages.append({"role": "user", "content": content})
        response = await self.client.chat(self.model, messages=self.messages)
        msg = response.message.model_dump() # pydantic
        self.messages.append(msg)
        return msg["content"]

client = Client()
response = await client.chat("Quand la France a-t-elle gagné la Coupe du Monde de football pour la première fois ?")
print(response)

La France a gagné la Coupe du Monde de football pour la première fois en **1998**. 

Ils ont battu l'équipe du Brésil en finale, sur le score de 3-0, lors du tournoi qui s'est déroulé en France.



In [13]:
import pprint
pprint.pp(client.messages)

[{'role': 'user',
  'content': 'Quand la France a-t-elle gagné la Coupe du Monde de football '
             'pour la première fois ?'},
 {'role': 'assistant',
  'content': 'La France a gagné la Coupe du Monde de football pour la première '
             'fois en **1998**. \n'
             '\n'
             "Ils ont battu l'équipe du Brésil en finale, sur le score de 3-0, "
             "lors du tournoi qui s'est déroulé en France.\n",
  'thinking': None,
  'images': None,
  'tool_calls': None}]


In [14]:
ollama.delete('gemma3n:latest')
ollama.list()

ListResponse(models=[])